In [2]:
%cd ..

/home/kuba/Projects/github_search


In [3]:
import requests
from io import StringIO
import sys
import time
import tqdm

import pypi_cli
from sklearn import feature_extraction, metrics
import numpy as np
import pandas as pd
import bs4

import mlutil.parallel

import elasticsearch
import haystack.document_store.memory
import haystack.document_store.elasticsearch
from haystack import document_store

import haystack.retriever.sparse
import haystack.retriever.dense
from haystack import retriever

from sklearn import pipeline


from github_search import repository_descriptions

03/06/2021 13:08:50 - INFO - faiss -   Loading faiss with AVX2 support.
03/06/2021 13:08:50 - INFO - faiss -   Loading faiss.
03/06/2021 13:08:51 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [4]:
!wc -l ../data/python/train.jsonl

wc: ../data/python/train.jsonl: No such file or directory


In [5]:
!head  -1 ../data/python/train.jsonl

head: cannot open '../data/python/train.jsonl' for reading: No such file or directory


In [6]:
all_codesearch_df = repository_descriptions.get_all_codesearch_df("data/python")

In [7]:
all_codesearch_df["repo"].value_counts().index[:20]

Index(['saltstack/salt', 'mitsei/dlkit', 'google/grr', 'bcbio/bcbio-nextgen',
       'materialsproject/pymatgen', 'tensorflow/tensor2tensor',
       'iotile/coretools', 'pandas-dev/pandas', 'cloud9ers/gurumate',
       'spyder-ide/spyder', 'pypa/pipenv', 'apple/turicreate', 'gem/oq-engine',
       'pantsbuild/pants', 'log2timeline/plaso',
       'googleapis/google-cloud-python', 'inasafe/inasafe', 'gwastro/pycbc',
       'apache/incubator-mxnet', 'senaite/senaite.core'],
      dtype='object')

In [8]:
repos_descriptions_df = repository_descriptions.load_pypi_repo_descriptions()

In [9]:
repos_descriptions_df.head()

,repo,pypi_description
Unnamed: 0,,
0,smdabdoub/phylotoast,Tools for phylogenetic data analysis including visualization and cluster-com...
1,mkouhei/bootstrap-py,Open-source algorithms for data-driven building analysis and control
2,elbow-jason/Uno-deprecated,Bootstrap Python package
3,disqus/nydus,Extremely fast and easy feature based HTML generator.
4,jay-johnson/network-pipeline,Connection utilities


In [10]:
all_codesearch_df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,smdabdoub/phylotoast,phylotoast/util.py,split_phylogeny,"def split_phylogeny(p, level=""s""):\n """"""\n Return either the full or t...",python,"def split_phylogeny(p, level=""s""):\n """"""\n Return either the full or t...","[def, split_phylogeny, (, p, ,, level, =, ""s"", ), :, level, =, level, +, ""__...",Return either the full or truncated version of a QIIME-formatted taxonomy st...,"[Return, either, the, full, or, truncated, version, of, a, QIIME, -, formatt...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
1,smdabdoub/phylotoast,phylotoast/util.py,ensure_dir,"def ensure_dir(d):\n """"""\n Check to make sure the supplied directory p...",python,"def ensure_dir(d):\n """"""\n Check to make sure the supplied directory p...","[def, ensure_dir, (, d, ), :, if, not, os, ., path, ., exists, (, d, ), :, t...","Check to make sure the supplied directory path does not exist, if so, create...","[Check, to, make, sure, the, supplied, directory, path, does, not, exist, if...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
2,smdabdoub/phylotoast,phylotoast/util.py,file_handle,"def file_handle(fnh, mode=""rU""):\n """"""\n Takes either a file path or a...",python,"def file_handle(fnh, mode=""rU""):\n """"""\n Takes either a file path or a...","[def, file_handle, (, fnh, ,, mode, =, ""rU"", ), :, handle, =, None, if, isin...","Takes either a file path or an open file handle, checks validity and returns...","[Takes, either, a, file, path, or, an, open, file, handle, checks, validity,...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
3,smdabdoub/phylotoast,phylotoast/util.py,gather_categories,"def gather_categories(imap, header, categories=None):\n """"""\n Find the...",python,"def gather_categories(imap, header, categories=None):\n """"""\n Find the...","[def, gather_categories, (, imap, ,, header, ,, categories, =, None, ), :, #...",Find the user specified categories in the map and create a dictionary to con...,"[Find, the, user, specified, categories, in, the, map, and, create, a, dicti...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
4,smdabdoub/phylotoast,phylotoast/util.py,parse_unifrac,"def parse_unifrac(unifracFN):\n """"""\n Parses the unifrac results file ...",python,"def parse_unifrac(unifracFN):\n """"""\n Parses the unifrac results file ...","[def, parse_unifrac, (, unifracFN, ), :, with, open, (, unifracFN, ,, ""rU"", ...",Parses the unifrac results file into a dictionary\n\n :type unifracFN: st...,"[Parses, the, unifrac, results, file, into, a, dictionary]",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train


## Baseline - retrieval by bag of words from descriptions

Fit bag of words model to descriptions, then match 'descriptions' obtained from concatenating comments from all functions from repository

The assumptions here are very optimistic, since we're only matching features from known repositories 

In [11]:
all_codesearch_df.columns

Index(['repo', 'path', 'func_name', 'original_string', 'language', 'code',
       'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url',
       'partition'],
      dtype='object')

In [12]:
all_codesearch_df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,smdabdoub/phylotoast,phylotoast/util.py,split_phylogeny,"def split_phylogeny(p, level=""s""):\n """"""\n Return either the full or t...",python,"def split_phylogeny(p, level=""s""):\n """"""\n Return either the full or t...","[def, split_phylogeny, (, p, ,, level, =, ""s"", ), :, level, =, level, +, ""__...",Return either the full or truncated version of a QIIME-formatted taxonomy st...,"[Return, either, the, full, or, truncated, version, of, a, QIIME, -, formatt...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
1,smdabdoub/phylotoast,phylotoast/util.py,ensure_dir,"def ensure_dir(d):\n """"""\n Check to make sure the supplied directory p...",python,"def ensure_dir(d):\n """"""\n Check to make sure the supplied directory p...","[def, ensure_dir, (, d, ), :, if, not, os, ., path, ., exists, (, d, ), :, t...","Check to make sure the supplied directory path does not exist, if so, create...","[Check, to, make, sure, the, supplied, directory, path, does, not, exist, if...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
2,smdabdoub/phylotoast,phylotoast/util.py,file_handle,"def file_handle(fnh, mode=""rU""):\n """"""\n Takes either a file path or a...",python,"def file_handle(fnh, mode=""rU""):\n """"""\n Takes either a file path or a...","[def, file_handle, (, fnh, ,, mode, =, ""rU"", ), :, handle, =, None, if, isin...","Takes either a file path or an open file handle, checks validity and returns...","[Takes, either, a, file, path, or, an, open, file, handle, checks, validity,...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
3,smdabdoub/phylotoast,phylotoast/util.py,gather_categories,"def gather_categories(imap, header, categories=None):\n """"""\n Find the...",python,"def gather_categories(imap, header, categories=None):\n """"""\n Find the...","[def, gather_categories, (, imap, ,, header, ,, categories, =, None, ), :, #...",Find the user specified categories in the map and create a dictionary to con...,"[Find, the, user, specified, categories, in, the, map, and, create, a, dicti...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
4,smdabdoub/phylotoast,phylotoast/util.py,parse_unifrac,"def parse_unifrac(unifracFN):\n """"""\n Parses the unifrac results file ...",python,"def parse_unifrac(unifracFN):\n """"""\n Parses the unifrac results file ...","[def, parse_unifrac, (, unifracFN, ), :, with, open, (, unifracFN, ,, ""rU"", ...",Parses the unifrac results file into a dictionary\n\n :type unifracFN: st...,"[Parses, the, unifrac, results, file, into, a, dictionary]",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train


In [13]:
all_codesearch_df.groupby("repo").agg("count")["url"]

repo
0101/pipetools                    8
02strich/django-auth-kerberos     1
05bit/peewee-async               30
0compute/xtraceback               2
0k/kids.cache                     2
                                 ..
zyga/padme                        2
zyga/python-glibc                24
zyga/python-phablet               2
zzzsochi/includer                 2
zzzsochi/resolver_deco            1
Name: url, Length: 12361, dtype: int64

In [14]:
len(repos_descriptions_df)

8940

In [15]:
codesearch_df = all_codesearch_df[
    all_codesearch_df["repo"].isin(repos_descriptions_df["repo"])
]

In [16]:
train_codesearch_df = codesearch_df[codesearch_df["partition"] == "train"]
val_codesearch_df = codesearch_df[codesearch_df["partition"] == "val"]

In [17]:
codesearch_df.partition.value_counts()

train    175471
test      11498
valid      9907
Name: partition, dtype: int64

In [18]:
set(all_codesearch_df["partition"].values)

{'test', 'train', 'valid'}

Note: we use Elasticsearch document store because in-memory tf-idf retriever is extremely slow.

Elasticsearch can be run using docker

`docker run -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.11.1`

In [19]:
def get_bm25_retriever(df, col, elasticsearch_client=elasticsearch.Elasticsearch()):
    docstring_store = document_store.elasticsearch.ElasticsearchDocumentStore(index=col)
    df["text"] = df[col]
    if not elasticsearch_client.count(index=col).get("count"):
        docstring_store.write_documents(df.to_dict("records"))
    return retriever.sparse.ElasticsearchRetriever(document_store=docstring_store)


def get_repos(ret, description, top_k=100):
    return [rec.meta["repo"] for rec in ret.retrieve(query=description, top_k=top_k)]

In [1]:
%%time
docstring_tfidf_retriever = get_bm25_retriever(codesearch_df, "docstring")

NameError: name 'get_bm25_retriever' is not defined

In [22]:
docstring_matched_repos = [
    get_repos(docstring_tfidf_retriever, descr)
    for descr in tqdm.tqdm(repos_descriptions_df["pypi_description"])
]

  4%|▍         | 373/8940 [00:07<02:51, 50.05it/s]

KeyboardInterrupt: 

In [23]:
len(docstring_matched_repos)

8940

In [24]:
recall_at100 = [
    repo in potential_repos
    for (repo, potential_repos) in zip(
        repos_descriptions_df["repo"], docstring_matched_repos
    )
]

In [25]:
np.mean(recall_at100)

0.007941834451901567

In [26]:
len(docstring_matched_repos)

8940

## Using Word Embeddings

In [30]:
project = neptune_util.init_neptune_project()

In [31]:
experiment = project.get_experiments()[0]

In [32]:
import2vec_file_name = "import2vec_module_vectors.bin"
experiment.download_artifact(import2vec_file_name, "/tmp")

In [33]:
import2vec_vectors = gensim.models.KeyedVectors.load(
    os.path.join("/tmp", import2vec_file_name)
)

In [33]:
def get_embedding_dim(embedder):
    if isinstance(embedder, pipeline.Pipeline):
        return embedder.steps[-1][1].components_.shape[0]
    elif isinstance(embedder, mlutil.feature_extraction.embeddings.EmbeddingVectorizer):
        return embedder.dimensionality_
    else:
        return embedder.components_.shape[0]


def make_dicts_with_added_field(dicts, field_name, field_values):
    for d, val in zip(dicts, field_values):
        d[field_name] = val
        yield d


def make_sklearn_retriever(
    df,
    pipeline,
    col="text",
    embedding_suffix="_emb",
    elasticsearch_client=elasticsearch.Elasticsearch(),
):
    embedding_dim = get_embedding_dim(pipeline)
    docstring_store = document_store.elasticsearch.ElasticsearchDocumentStore(
        index=col, embedding_field=col + embedding_suffix, embedding_dim=embedding_dim
    )
    df["text"] = df[col]
    embeddings = pipeline.fit_transform(df["text"])
    documents = df.to_dict("records")
    dicts_with_added_fields = list(
        make_dicts_with_added_field(documents, col + embedding_suffix, embeddings)
    )
    if not elasticsearch_client.count(index=col).get("count"):
        docstring_store.write_documents(tqdm.tqdm(dicts_with_added_fields))
    return retriever.dense.SklearnTransformerRetriever(
        embedding_transformer=pipeline, document_store=docstring_store
    )

In [35]:
from mlutil.feature_extraction import embeddings

keyed_vectors = mlutil.feature_extraction.embeddings.load_gensim_embedding_model(
    "glove-wiki-gigaword-100"
)

In [36]:
transformer_vectorizer = embeddings.AverageWordEmbeddingsVectorizer(keyed_vectors)
transformer_vectorizer.dimensionality_

100

In [37]:
%%time
transformer_vectorizer.fit(codesearch_df["text"])

CPU times: user 96 µs, sys: 16 µs, total: 112 µs
Wall time: 113 µs


In [51]:
codesearch_df.partition.value_counts()

train    175471
test      11498
valid      9907
Name: partition, dtype: int64

In [30]:
%%time
codesearch_df["test_text"] = codesearch_df["text"]
average_embeddings_dense_retriever = make_sklearn_retriever(
    codesearch_df, transformer_vectorizer, "test_text"
)

<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


NameError: name 'transformer_vectorizer' is not defined

In [50]:
docstring_embedding_matched_repos = [
    get_repos(average_embeddings_dense_retriever, descr)
    for descr in tqdm.tqdm(repos_descriptions_df["pypi_description"])
]

100%|██████████| 8940/8940 [08:15<00:00, 18.03it/s]


In [52]:
codesearch_df["text"]

0        Return either the full or truncated version of a QIIME-formatted taxonomy st...
1        Check to make sure the supplied directory path does not exist, if so, create...
2        Takes either a file path or an open file handle, checks validity and returns...
3        Find the user specified categories in the map and create a dictionary to con...
4        Parses the unifrac results file into a dictionary\n\n    :type unifracFN: st...
                                              ...                                       
14913    Return a canvas from a grayscale image.\n\n        :param im: single channel...
14914    Returns a unique ID of a given length.\n    User `version=2` for cross-syste...
14915                                                   Build a unique key from get data
14916                                               Returns domain name portion of a URL
14917                                             Returns a dictionary from a URL params
Name: text, Length: 1

In [53]:
recall_at100 = [
    repo in potential_repos
    for (repo, potential_repos) in zip(
        repos_descriptions_df["repo"], docstring_embedding_matched_repos
    )
]

In [54]:
np.mean(recall_at100)

0.008277404921700223

## Doc2Vec embeddings

In [26]:
from github_search import neptune_util
import gensim

In [27]:
project = neptune_util.init_neptune_project()
experiment = project.get_experiments()[1]
from mlutil import feature_extraction

In [30]:
model_path = "/tmp/doc2vec_description_comments.bin"
experiment.download_artifact("doc2vec_description_comments.bin", "/tmp")

In [31]:
%%time
codesearch_df["d2v_test_text"] = codesearch_df["docstring"]
doc2vec_model = gensim.models.Doc2Vec.load(model_path)
doc2vectorizer = feature_extraction.embeddings.Doc2Vectorizer(doc2vec_model)

<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
03/03/2021 21:06:03 - INFO - gensim.utils -   loading Doc2Vec object from /tmp/doc2vec_description_comments.bin
03/03/2021 21:06:03 - INFO - gensim.utils -   loading vocabulary recursively from /tmp/doc2vec_description_comments.bin.vocabulary.* with mmap=None
03/03/2021 21:06:03 - INFO - gensim.utils -   loading trainables recursively from /tmp/doc2vec_description_comments.bin.trainables.* with mmap=None
03/03/2021 21:06:03 - INFO - gensim.utils -   loading wv recursively from /tmp/doc2vec_description_comments.bin.wv.* with mmap=None
03/03/2021 21:06:03 - INFO - gensim.utils -   loading docvecs recursively from /tmp/doc2vec_description_comments.bin.docvecs.* with mmap=None
03/03/2021 21:0

CPU times: user 143 ms, sys: 19.6 ms, total: 163 ms
Wall time: 162 ms


In [35]:
%%time
doc2vec_dense_retriever = make_sklearn_retriever(
    codesearch_df, doc2vectorizer, "d2v_test_text"
)

03/03/2021 21:06:58 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.005s]
03/03/2021 21:06:58 - INFO - elasticsearch -   PUT http://localhost:9200/d2v_test_text [status:200 request:0.092s]
03/03/2021 21:06:58 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.002s]
<ipython-input-33-825d834b9757>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df[col]
03/03/2021 21:08:03 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_count [status:200 request:0.006s]
  3%|▎         | 5515/196876 [00:00<00:03, 55144.53it/s]03/03/2021 21:08:05 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.174s]
03/03/2021 21:08:06 - IN

03/03/2021 21:09:07 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.034s]
03/03/2021 21:09:08 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.017s]
03/03/2021 21:09:09 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.037s]
 19%|█▉        | 37470/196876 [01:06<04:11, 634.99it/s]03/03/2021 21:09:10 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.874s]
03/03/2021 21:09:11 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.014s]
03/03/2021 21:09:13 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.030s]
03/03/2021 21:09:14 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.018s]
03/03/2021 21:09:15 - INFO - elasticsearch -   POST http://localhost:9

 34%|███▍      | 67803/196876 [02:12<03:31, 611.30it/s]03/03/2021 21:10:16 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.879s]
03/03/2021 21:10:18 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.047s]
03/03/2021 21:10:19 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.013s]
03/03/2021 21:10:20 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.011s]
03/03/2021 21:10:21 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.004s]
03/03/2021 21:10:22 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.017s]
03/03/2021 21:10:23 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.009s]
03/03/2021 21:10:24 - INFO - elasticsearch -   POST http://localhost:9

03/03/2021 21:11:25 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.028s]
03/03/2021 21:11:26 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.022s]
03/03/2021 21:11:27 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.022s]
03/03/2021 21:11:28 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.013s]
03/03/2021 21:11:29 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.006s]
03/03/2021 21:11:30 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.020s]
03/03/2021 21:11:31 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.031s]
03/03/2021 21:11:33 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.044s]


03/03/2021 21:12:34 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.012s]
03/03/2021 21:12:36 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.078s]
03/03/2021 21:12:37 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.021s]
03/03/2021 21:12:38 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.976s]
03/03/2021 21:12:39 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.021s]
03/03/2021 21:12:40 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.042s]
03/03/2021 21:12:41 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.016s]
03/03/2021 21:12:42 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.019s]


 80%|████████  | 158033/196876 [05:40<01:03, 611.53it/s]03/03/2021 21:13:44 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.010s]
03/03/2021 21:13:45 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.989s]
03/03/2021 21:13:46 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.032s]
03/03/2021 21:13:47 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.030s]
03/03/2021 21:13:48 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.020s]
03/03/2021 21:13:49 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.006s]
03/03/2021 21:13:50 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.007s]
03/03/2021 21:13:51 - INFO - elasticsearch -   POST http://localhost:

03/03/2021 21:14:52 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.046s]
 95%|█████████▍| 186710/196876 [06:50<00:17, 572.37it/s]03/03/2021 21:14:54 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.039s]
03/03/2021 21:14:55 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.025s]
03/03/2021 21:14:56 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.012s]
03/03/2021 21:14:57 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.023s]
03/03/2021 21:14:58 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.053s]
03/03/2021 21:14:59 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.039s]
03/03/2021 21:15:00 - INFO - elasticsearch -   POST http://localhost:

CPU times: user 1min 36s, sys: 1.04 s, total: 1min 37s
Wall time: 8min 17s


In [64]:
docstring_doc2vec_matched_repos = [
    get_repos(doc2vec_dense_retriever, descr, top_k=200)
    for descr in tqdm.tqdm(repos_descriptions_df["pypi_description"])
]

  1%|          | 47/8940 [00:06<20:44,  7.14it/s]03/03/2021 21:49:45 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.111s]


  1%|          | 95/8940 [00:13<21:05,  6.99it/s]03/03/2021 21:49:52 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.127s]


  2%|▏         | 189/8940 [00:26<20:52,  6.99it/s]03/03/2021 21:50:05 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.111s]


  3%|▎         | 283/8940 [00:40<20:32,  7.02it/s]03/03/2021 21:50:19 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.124s]


  4%|▍         | 377/8940 [00:53<20:21,  7.01it/s]03/03/2021 21:50:32 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.114s]


...

 98%|█████████▊| 8743/8940 [20:32<00:28,  7.03it/s]03/03/2021 22:10:11 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.116s]


 98%|█████████▊| 8790/8940 [20:39<00:21,  7.08it/s]03/03/2021 22:10:18 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.115s]


 99%|█████████▉| 8837/8940 [20:46<00:14,  7.08it/s]03/03/2021 22:10:25 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.111s]


 99%|█████████▉| 8884/8940 [20:52<00:07,  7.14it/s]03/03/2021 22:10:31 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.112s]


100%|█████████▉| 8900/8940 [20:55<00:05,  7.29it/s]03/03/2021 22:10:33 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.075s]
03/03/2021 22:10:34 - INFO - elasticsearch -   POST http://localhost:9200/d2v_test_text/_search [status:200 request:0.116s]
100%|██████████| 8940/8940 [21:00<00:00,  7.09it/s]


In [66]:
doc2vec_recall_at100 = [
    repo in pd.Series(potential_repos).unique()[:100]
    for (repo, potential_repos) in zip(
        repos_descriptions_df["repo"], docstring_doc2vec_matched_repos
    )
]

In [65]:
doc2vec_recall_at50 = [
    repo in pd.Series(potential_repos).unique()[:50]
    for (repo, potential_repos) in zip(
        repos_descriptions_df["repo"], docstring_doc2vec_matched_repos
    )
]

In [ ]:
np.mean(recall_at100)

In [47]:
np.mean(doc2vec_recall_at100)

0.007494407158836689

In [51]:
retrieved_repos_counts = [len(set(repos)) for repos in docstring_doc2vec_matched_repos]

In [63]:
np.mean(doc2vec_recall_at50)

0.006935123042505592

In [49]:
len(repos_descriptions_df)

8940